## Allele filp QC YML generator
This module takes in a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr and generate a list of yml to be used

## Example target list

#chr    TARGET
1    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr1.txt
10    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr10.txt
11    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr11.txt
12    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr12.txt
13    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr13.txt
14    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr14.txt
15    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr15.txt
16    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr16.txt
17    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr17.txt
18    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr18.txt
19    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr19.txt
2    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr2.txt
20    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr20.txt
21    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr21.txt
22    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr22.txt
3    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr3.txt
4    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr4.txt
5    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr5.txt
6    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr6.txt
7    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr7.txt
8    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr8.txt
9    /mnt/mfs/statgen/snuc_pseudo_bulk/eight_tissue_analysis/8test/TARGET_ref_chr9.txt

In [1]:
[global]
# List of path to the index of sumstat, each correspond to 1 recipe file documenting the path to the sumstat of each chromosome.
parameter: sumstat_list = paths
# List of names that corresponding to each of the studies
parameter: name = list
parameter: cwd = path
import pandas as pd
input_list = sumstat_list
sumstat_list = pd.read_csv(input_list[0],sep = "\t")
sumstat_list = sumstat_list.sort_values('#chr')
if len(input_list) > 1:
    for x in range(1,len(input_list)):
        sumstat_list = sumstat_list.merge(pd.read_csv(input_list[x],sep = "\t"), on = "#chr")
sumstat_meta = sumstat_list.filter(regex='column').iloc[0].values.tolist()
sumstat_list = sumstat_list.drop(sumstat_list.filter(regex='column').columns,axis = 1)
sumstat_list.columns = ["#chr"] + name
sumstat_inv = sumstat_list.values.tolist()
names = "_".join(name)
## The target vcf file, with GENE,CHR,POS,A0,A1 as columns, should contains all snps
parameter: TARGET_list = path("./")
if TARGET_list.is_file():
    sumstat_inv = sumstat_list.merge(pd.read_csv(TARGET_list,sep = "\t"), on = "#chr" ).values.tolist()
## Assuming all the input sumstat are using the same header
parameter: CHR = "chrom"
parameter: POS = "pos"
parameter: A0 = "ref" 
parameter: A1 = "alt"
parameter: SNP = "variant_id"
parameter: STAT = "beta"
parameter: SE = "se"
parameter: P = "pval"
parameter: KEEP_AMBIGUOUS = "True"
parameter: GENE = "phenotype_id"
parameter: container = ""

In [ ]:
[yml_generator]
input: for_each  = "sumstat_inv"
output: f'{cwd:a}/{names}.{_sumstat_inv[0]}/{names}.{_sumstat_inv[0]}.yml'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout' , container = container
    import os
    import yaml
    import pandas as pd
    output = dict()
    ## Input dict
    output["INPUT"] = [pd.read_csv(y,"\t",index_col = 0,names = [x],header = 0 ).to_dict() for x,y in zip($[_sumstat_inv[1:len(_sumstat_inv)-1] if TARGET_list.is_file() else  _sumstat_inv[1:len(_sumstat_inv)]],$[sumstat_meta])]       
    ## Target dict
    output["TARGET"] = ["$[_sumstat_inv[-1] if TARGET_list.is_file() else _sumstat_inv[1]]"] ## Allow for both external TARGET file or using one of the sumstat as TARGET
    ## Output dict
    output["OUTPUT"] = [$[_output:dr]]
    with open($[_output:ar], 'w') as f:
        yaml.dump(output,f,sort_keys = False)

In [ ]:
[yml_list]
input: output_from("yml_generator"),group_by = "all"
output: f'{cwd:a}/yml_list.txt', f'{cwd:a}/qced_sumstat_list.txt'
import pandas as pd
yml_df = pd.DataFrame({"#chr" : sumstat_list["#chr"].values.tolist()  ,  "dir" : _input})
yml_df.to_csv(_output[0],sep = "\t",index = 0)
data_dir_tmp = pd.Series(_input)
data_dir = [f'{x:d}' for x in data_dir_tmp ]
theme = sumstat_list.columns.values.tolist()[1:]
for i in theme:
    sumstat_list = sumstat_list.assign(**{i : data_dir+ pd.Series([f'/{path(x):b}'  for x in  sumstat_list[i].values.tolist()])  } )
sumstat_list.to_csv(_output[1],sep = "\t", index = 0)